In [1]:
# Import library for Flask
from flask import Flask, jsonify
from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from

# Import library or framework for Machine Learning process
import pickle, re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# DEFINE SWAGGER UI DESCRIPTION
app = Flask(__name__)
app.json_provider_class = LazyJSONEncoder
app.json = LazyJSONEncoder(app)
swagger_template = dict(
info = {
    'title': LazyString(lambda: 'API Documentation for LSTM'),
    'version': LazyString(lambda: '1.0.0'),
    'description': LazyString(lambda: 'Dokumentasi API untuk LSTM'),
    },
    host = LazyString(lambda: request.host)
)
swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'docs',
            "route": '/docs.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}
swagger = Swagger(app, template=swagger_template,             
                  config=swagger_config)

# Define Feature Extraction parameter and Tokenizer Class
max_features = 100000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)

# Define sentiments label
sentiment = ['negative', 'neutral', 'positive']

# Cleansing process
def cleansing(sent):
    # Mengubah kata menjadi huruf kecil semua dengan menggunakan fungsi lower()
    string = sent.lower()
    # Menghapus emoticon dan tanda baca menggunakan "RegEx" dengan script di bawah
    string = re.sub(r'[^a-zA-Z0-9]', ' ', string)
    return string

# LSTM
# Load result of Feature Extraction process from LSTM
file = open('C:/Users/Faza/Documents/Binar/Challenge Platinum/23001012-13-faz-sentiment-platinum/resources_of_lstm/x_pad_sequences.pickle', 'rb')
feature_file_from_lstm = pickle.load(file)
file.close()

# Load model from LSTM
model_file_from_lstm = load_model('C:/Users/Faza/Documents/Binar/Challenge Platinum/23001012-13-faz-sentiment-platinum/model_of_lstm/model.h5')

# Neural Network
## Load result of Feature Extraction process from NN
file = open('C:/Users/Faza/Documents/Binar/Challenge Platinum/23001012-13-faz-sentiment-platinum/resources_of_nn/feature.p', 'rb')
feature_file_from_nn = pickle.load(file)
file.close()

## Load model from NN
model_file_from_nn = pickle.load(open('C:/Users/Faza/Documents/Binar/Challenge Platinum/23001012-13-faz-sentiment-platinum/model_of_nn/model.p', 'rb'))

# DEFINE ENDPOINTS: POST SENTIMENT ANALYSIS USING LSTM FROM TEXT
@swag_from("C:/Users/Faza/Documents/Binar/Challenge Platinum/23001012-13-faz-sentiment-platinum/docs/lstm_text_processing.yml", methods=['POST'])
@app.route('/lstm-text-processing',methods=['POST'])
def lstm_text_processing():
    # Get text
    original_text = request.form.get('text')
    # Cleansing
    text = [cleansing(original_text)]
    # Feature Extraction
    predicted = tokenizer.texts_to_sequences(text)
    guess = pad_sequences(predicted, maxlen=feature_file_from_lstm.shape[1])
    # Inference
    prediction = model_file_from_lstm.predict(guess)
    get_sentiment = sentiment[np.argmax(prediction[0])]
    
    # Define API Response
    json_response = {
        'status_code':200,
        'description':'Result of Sentiment Analysis using LSTM',
        'data' : {
            'text' : original_text,
            'sentiment' : get_sentiment
        },
    }
    response_data=jsonify(json_response)
    return response_data

# DEFINE ENDPOINTS: POST SENTIMENT ANALYSIS USING LSTM FROM FILE
@swag_from("C:/Users/Faza/Documents/Binar/Challenge Platinum/23001012-13-faz-sentiment-platinum/docs/lstm_file_processing.yml", methods=['POST'])
@app.route('/lstm-file-processing', methods=['POST'])
def lstm_file_processing():

    # Uploaded file
    file = request.files.getlist('file')[0]

    # Import file csv to Pandas
    df = pd.read_csv(file, encoding='latin-1')

    # Get text from file in "List" format
    texts = df.Tweet.to_list()

    # Loop list or original text and predict to model
    text_with_sentiment = []
    for original_text in texts:

        # Cleansing
        text = [cleansing(original_text)]
        # Feature extraction
        predicted = tokenizer.texts_to_sequences(text)
        guess = pad_sequences(predicted, maxlen=feature_file_from_lstm.shape[1])
        # Inference
        prediction = model_file_from_lstm.predict(guess)
        get_sentiment = sentiment[np.argmax(prediction[0])]

        # Predict "text_clean" to the Model. And insert to list "text_with_sentiment".
        text_with_sentiment.append({
            'text': original_text,
            'sentiment': get_sentiment
        })
       
    # Define API response
    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': text_with_sentiment,
    }
    response_data = jsonify(json_response)
    return response_data

# DEFINE ENDPOINTS: POST SENTIMENT ANALYSIS USING NN FROM TEXT
@swag_from("C:/Users/Faza/Documents/Binar/Challenge Platinum/23001012-13-faz-sentiment-platinum/docs/nn_text_processing.yml", methods=['POST'])
@app.route('/nn-text-processing', methods=['POST'])
def nn_text_processing():
    
    # Get text
    original_text = request.form.get('text')
    # Cleansing
    text = cleansing(original_text)
    # Feature extraction
    text_feature = feature_file_from_nn.transform([text])
    # Inference
    get_sentiment = model_file_from_nn.predict(text_feature)[0]
    
    # Define API response
    json_response = {
        'status_code': 200,
        'description': "Result of Sentiment Analysis using CNN",
        'data': {
            'text': original_text,
            'sentiment': get_sentiment
        },
    }
    response_data = jsonify(json_response)
    return response_data

# DEFINE ENDPOINTS: POST SENTIMENT ANALYSIS USING NN FROM FILE
@swag_from("C:/Users/Faza/Documents/Binar/Challenge Platinum/23001012-13-faz-sentiment-platinum/docs/nn_file_processing.yml", methods=['POST'])
@app.route('/nn-file-processing', methods=['POST'])
def nn_file_processing():

    # Uploaded file
    file = request.files.getlist('file')[0]

    # Import file csv to Pandas
    df = pd.read_csv(file, encoding='latin-1')

    # Get text from file in "List" format
    texts = df.Tweet.to_list()

    # Loop list or original text and predict to model
    text_with_sentiment = []
    for original_text in texts:

        # Cleansing
        text = cleansing(original_text)
        # Feature extraction
        text_feature = feature_file_from_nn.transform([text])
        # Inference
        get_sentiment = model_file_from_nn.predict(text_feature)[0]

        # Predict "text_clean" to the Model. And insert to list "text_with_sentiment".
        text_with_sentiment.append({
            'text': original_text,
            'sentiment': get_sentiment
        })
    
    # Define API response
    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': text_with_sentiment,
    }
    response_data = jsonify(json_response)
    return response_data

if __name__ == "__main__":
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Dec/2023 22:35:11] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Dec/2023 22:35:13] "GET /docs/ HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2023 22:35:13] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Dec/2023 22:35:13] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Dec/2023 22:35:13] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Dec/2023 22:35:13] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Dec/2023 22:35:13] "GET /docs.json HTTP/1.1" 200 -


1/1 [==============================] - 0s 404ms/step


127.0.0.1 - - [03/Dec/2023 22:35:20] "POST /lstm-text-processing HTTP/1.1" 200 -


1/1 [==============================] - 0s 21ms/step


...

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 19ms/step


127.0.0.1 - - [03/Dec/2023 22:47:17] "POST /lstm-file-processing HTTP/1.1" 200 -
